### __Implement a hyperparameter optimization strategy (random search) for a neural network model.__

In [1]:
import numpy as np
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import tensorflow as tf

In [2]:
df = sns.load_dataset('iris')
df.head()

x = df.drop(columns=['sepal_length'])
y = df['sepal_length']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

le = LabelEncoder()
x_train['species'] = le.fit_transform(x_train['species'])
x_test['species']= le.transform(x_test['species'])

In [3]:
def test_model(hp):
    model=Sequential()
    model.add(Input(shape=(x_train.shape[1],)))
    model.add(Dense(hp.Int('unit1',min_value=32,max_value=128),activation=hp.Choice('activation1',['relu','tanh','sigmoid'])))
    model.add(Dense(hp.Int('unit2',min_value=32,max_value=128),activation=hp.Choice('activation2',['relu','tanh','sigmoid'])))
    model.add(Dense(1))

    model.compile(optimizer=SGD(hp.Float('optimize',min_value=0.00001,max_value=0.001),momentum=hp.Float('momentum',min_value=0.0,max_value=0.9)),loss='mean_squared_error',metrics=['mae'])

    return model
    

In [5]:
tuner = kt.RandomSearch(
    test_model,
    objective='val_mae',  
    max_trials=10,
    executions_per_trial=1, 
    directory='test', 
    project_name='iris_reg3' 
)

In [6]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hp)
best_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
loss, mae = best_model.evaluate(x_test, y_test)
print(f"Test MAE: {mae}")

Trial 10 Complete [00h 00m 04s]
val_mae: 0.7491304874420166

Best val_mae So Far: 0.4718247652053833
Total elapsed time: 00h 00m 47s
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - loss: 45.0135 - mae: 6.6568 - val_loss: 38.9392 - val_mae: 6.2007
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 34.3431 - mae: 5.8140 - val_loss: 25.1013 - val_mae: 4.9918
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 21.4889 - mae: 4.6058 - val_loss: 14.6265 - val_mae: 3.8072
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 12.2260 - mae: 3.4536 - val_loss: 7.3933 - val_mae: 2.6658
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 6.2513 - mae: 2.3945 - val_loss: 3.1576 - val_mae: 1.5931
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 2.5705 - mae: 1.3338 - val_loss: 1.5589 - val_mae: 0.9283
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 1.3325 - mae: 0.9177 - val_loss: 1.3185 - val_mae: 0.9723
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - los